In [ ]:
from Metaheuristicas.fitness_functions import *
import json

import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_classif, chi2
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Necessary functions

### Fitness Functions

In [ ]:

from skrebate import ReliefF


def mutual_information_eval(solution, data, labels):
    # Convert NumPy array back to DataFrame
    data_df = pd.DataFrame(data)
    selected_data = data_df.iloc[:, solution == 1]
    if selected_data.shape[1] == 0:
        return -np.inf
    mi_scores = mutual_info_classif(selected_data, labels)
    return np.sum(mi_scores)


def chi2_eval(solution, data, labels):
    # Convert NumPy array back to DataFrame
    data_df = pd.DataFrame(data)
    selected_data = data_df.iloc[:, solution == 1]
    if selected_data.shape[1] == 0:
        return -np.inf
    chi2_scores, _ = chi2(selected_data, labels)
    return np.mean(chi2_scores)





def relieff_eval(solution, data, labels, n_neighbors=10):
    
    if not isinstance(data, pd.DataFrame):
        data = pd.DataFrame(data)

    selected_features = data.iloc[:, solution.astype(bool)]

    if selected_features.shape[1] == 0:
        return -np.inf

    labels = np.array(labels)

    relief = ReliefF(n_neighbors=n_neighbors)
    relief.fit(selected_features.values, labels)

    relieff_score = relief.feature_importances_.mean()

    return relieff_score

def load_and_preprocess_data(filename='Resources/SeisBenchV1_v1_1.json'):
    """
    Load and preprocess data from a JSON file.

    Parameters:
    - filename: Path to the JSON file.

    Returns:
    - X_scaled: Scaled feature matrix (DataFrame).
    - y: Target variable.
    """
    with open(filename) as file:
        data = json.load(file)
        data = pd.DataFrame(data)
        data.dropna(inplace=True)
        data.drop(data[data['Type'] == 'REGIONAL'].index, inplace=True)
        data.drop(data[data['Type'] == 'HB'].index, inplace=True)
        data.drop(data[data['Type'] == 'ICEQUAKE'].index, inplace=True)
        data.drop(data[data['Type'] == ''].index, inplace=True)

    label_encoder = LabelEncoder()
    data['Type'] = label_encoder.fit_transform(data['Type'])

    X = data.iloc[:, 1:]
    y = data['Type']

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    return pd.DataFrame(X_scaled, columns=X.columns), y







In [ ]:
def add_result(classifier, fitness_function, accuracy, precision, recall, f1_score, auc):
    new_data = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score,
        "AUC": auc
    }

    if classifier == "Naive Bayes":
        global naive_bayes_df
        naive_bayes_df.loc[fitness_function] = new_data
    elif classifier == "Random Forest":
        global random_forest_df
        random_forest_df.loc[fitness_function] = new_data
    elif classifier == "Neural Network":
        global neural_network_df
        neural_network_df.loc[fitness_function] = new_data

### Save Results

In [ ]:
from IPython.display import display, clear_output

# Step 1: Initialize empty DataFrames for each classifier with metrics as columns
metrics = ["Accuracy", "Precision", "Recall", "F1 Score", "AUC"]

naive_bayes_df = pd.DataFrame(columns=metrics, index=["Mutual Information", "X2", "Relief"])
random_forest_df = pd.DataFrame(columns=metrics, index=["Mutual Information", "X2", "Relief"])
neural_network_df = pd.DataFrame(columns=metrics, index=["Mutual Information", "X2", "Relief"])

# Display all tables function
def display_tables():
    clear_output(wait=True)
    print("Naive Bayes Results")
    display(naive_bayes_df)
    print("Random Forest Results")
    display(random_forest_df)
    print("Neural Network Results")
    display(neural_network_df)

# Cuckoo Search implementation

In [ ]:
import json

from sklearn.feature_selection import mutual_info_classif

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from scipy.special import gamma

from Metaheuristicas.fitness_functions import mutual_information_eval, relieff_eval, chi2_eval
from Metaheuristicas.fitness_functions import load_and_preprocess_data



# Función para realizar vuelos de Levy
def levy_flight(Lambda=1.5, size=1):
    sigma = (gamma(1 + Lambda) * np.sin(np.pi * Lambda / 2) /
             (gamma((1 + Lambda) / 2) * Lambda * 2 ** ((Lambda - 1) / 2))) ** (1 / Lambda)
    u = np.random.normal(0, sigma, size)
    v = np.random.normal(0, 1, size)
    step = u / abs(v) ** (1 / Lambda)
    return step


# Búsqueda de cucos con impresiones de seguimiento
def cuckoo_search(n, dim, iter_max, data, labels, pa=0.25, fitness_function=mutual_information_eval):
    # Initialize nests as random binary matrices (n nests, each with 'dim' features)
    nests = np.random.rand(n, dim) > 0.5

    # Calculate the initial fitness of all nests using mutual information
    fitness = np.array([fitness_function(nest.astype(int), data, labels) for nest in nests])

    # Main loop: iterate for 'iter_max' iterations
    for t in range(iter_max):
        # print(f"Iteración {t + 1}/{iter_max}")  # Display current iteration number

        # For each nest, perform the Cuckoo Search process
        for i in range(n):
            # Generate a new solution via Levy flight (binary mutation)
            step_size = levy_flight(size=dim) > 0.5  # Create a random step using Levy flight
            new_nest = np.logical_xor(nests[i], step_size).astype(int)  # Mutate the current nest

            # Evaluate the fitness of the newly generated nest
            new_fitness = fitness_function(new_nest, data, labels)
            # print(f"  Nido {i + 1} fitness: {fitness[i]:.4f} -> {new_fitness:.4f}")
            # Select a random different nest to compare the new solution against
            random_nest_index = np.random.choice([j for j in range(n) if j != i])

            # If the new nest has better fitness than the random nest, replace it
            if new_fitness > fitness[random_nest_index]:
                nests[random_nest_index] = new_nest  # Replace the random nest with the new one
                fitness[random_nest_index] = new_fitness  # Update fitness of the replaced nest
                # print(f"  Nido {random_nest_index + 1} mejorado a fitness {new_fitness:.4f}")

        # Abandon a fraction 'pa' of the worst nests and replace them with new random nests
        n_abandon = int(n * pa)  # Calculate the number of nests to abandon
        worst_nests_indices = np.argsort(fitness)[:n_abandon]  # Indices of the worst nests

        # Replace the worst nests with new random ones and recalculate their fitness
        nests[worst_nests_indices] = np.random.rand(n_abandon, dim) > 0.5
        fitness[worst_nests_indices] = np.array(
            [fitness_function(nest.astype(int), data, labels) for nest in nests[worst_nests_indices]]
        )
        # print(f"  {n_abandon} peores nidos abandonados y reemplazados por nuevos nidos.")

        # Display the best fitness found at the end of this iteration
        best_fitness = np.max(fitness)
        print(f"Mejor fitness al final de la iteración {t + 1}: {best_fitness:.4f}\n")

    # After all iterations, find the best solution (nest) with the highest fitness
    best_idx = np.argmax(fitness)  # Index of the best nest
    best_nest = nests[best_idx]  # The best nest (binary feature selection vector)
    best_fitness = fitness[best_idx]  # The best fitness value
    # print(f"Mejor nido encontrado:\n{best_nest}\nCon un fitness de: {best_fitness:.4f}")

    # Return the final set of nests, their fitness values, and the best solution
    return nests, fitness, best_nest, best_fitness


In [ ]:
X, y = load_and_preprocess_data(filename='Resources/SeisBenchV1_v1_1.json')


# Optimization

In [ ]:
#Save the results from the optimizations
Mutual_Information = []
Chi2 = []
ReliefFList = []

In [ ]:

n = 40
dim = 84

In [ ]:
for i in range(100,550,50):
    nests, fitness_scores, best_nest, best_CKMIfitness = cuckoo_search(n, dim, i, X, y, fitness_function=mutual_information_eval)
    CKMIFeatures = X.columns[best_nest.astype(bool)].tolist()

    Mutual_Information.append([CKMIFeatures, best_CKMIfitness, i])

In [ ]:
for i in range(100,550,50):
    nests, fitness_scores, best_nest, best_CX2fitness = cuckoo_search(n, dim, i, X, y, fitness_function=chi2_eval)
    CKX2Features = X.columns[best_nest.astype(bool)].tolist()

    Chi2.append([CKX2Features, best_CX2fitness, i])

In [ ]:
for i in range(100,550,50):
    nests, fitness_scores, best_nest, best_CKRFfitness = cuckoo_search(n, dim, i, X, y, fitness_function=relieff_eval)
    CKRFFeatures = X.columns[best_nest.astype(bool)].tolist()

    ReliefFList.append([CKRFFeatures, best_CKRFfitness, i])

In [ ]:
# import pickle
# with open('SAMutual_Information.pkl', 'wb') as f:
#     pickle.dump(Mutual_Information, f)
# with open('SAChi2.pkl', 'wb') as f:
#     pickle.dump(Chi2, f)
# with open('SAReliefF.pkl', 'wb') as f:
#     pickle.dump(ReliefF, f)

In [ ]:
Mutual_Information.sort(key=lambda x: x[1], reverse=True)
Chi2.sort(key=lambda x: x[1], reverse=True)
ReliefFList.sort(key=lambda x: x[1], reverse=True)

In [ ]:
for i in Mutual_Information:
    print("Mutual Information: ",i[1], i[2])

In [ ]:
for i in Chi2:
    print("Chi2: ",i[1], i[2])

In [ ]:
for i in ReliefFList:
    print("ReliefF List: ",i[1], i[2])

In [ ]:
print("Mutual Information selected Features: \n",len(Mutual_Information[0][0]))


In [ ]:
print("Chi2 selected features: \n",len(Chi2[0][0]))

In [ ]:
print("ReliefF selected features: \n",len(ReliefFList[0][0]))


## Base Results

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Naive Bayes

In [ ]:


#Naive Bayes classifier with all features

cmodel = GaussianNB()
cmodel.fit(X_train, y_train)
y_pred = cmodel.predict(X_test)
y_pred_proba = cmodel.predict_proba(X_test)[:, 1]  # Probability estimates for AUC

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

In [ ]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("AUC: ", auc)


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest.fit(X_train, y_train)
y_pred = random_forest.predict(X_test)
y_pred_proba = random_forest.predict_proba(X_test)[:, 1]  # Probability estimates for AUC
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)


In [ ]:
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("AUC: ", auc)


### Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:


# Crear y compilar el modelo
RN = Sequential()

# Definir la dimensión de entrada basada en las características seleccionadas
input_dim = X_train.shape[1]
RN.add(Dense(units=64, activation='relu', input_dim=input_dim))
RN.add(Dense(units=32, activation='relu'))
RN.add(Dense(units=1, activation='sigmoid'))

# Compilar el modelo
RN.compile(optimizer=Adam(learning_rate=0.001), 
              loss=BinaryCrossentropy(),
              metrics=['accuracy'])

# Entrenar el modelo
history = RN.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = RN.evaluate(X_test, y_test)

print(f'Test Accuracy: {accuracy}')

# Realizar predicciones en el conjunto de prueba
y_pred_prob = RN.predict(X_test)  # Predicciones como probabilidades
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades a etiquetas (0 o 1)




In [ ]:
# Calcular y mostrar las métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_prob))  # Usamos probabilidades para AUC
add_result("Neural Network", "Mutual Information", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred_prob))

In [ ]:
MI_Scores = []
Chi_Scores = []
ReliefF_Scores = []

# Cuckoo search with optimized parameters (best fitness)

In [ ]:
MIOpt= []
ChiOpt = []
ReliefOpt = []

In [ ]:
nests, fitness_scores, best_nest, best_CKMIfitness = cuckoo_search(n, dim, Mutual_Information[0][1], X, y, fitness_function=mutual_information_eval)

CKMIFeatures = X.columns[best_nest.astype(bool)].tolist()
MIOpt.append([CKMIFeatures, best_CKMIfitness])

In [ ]:
print(MIOpt)

In [ ]:
print("Features selected by MI: ", CKMIFeatures)


In [ ]:
nests, fitness_scores, best_nest, best_CX2fitness = cuckoo_search(n, dim, Chi2[0][1], X, y, fitness_function=chi2_eval)
CKX2Features = X.columns[best_nest.astype(bool)].tolist()
ChiOpt.append([CKX2Features, best_CX2fitness])

In [ ]:
print(ChiOpt)

In [ ]:
print("Features selected by Chi2: ", CKX2Features)

In [ ]:
nests, fitness_scores, best_nest, best_CKRFfitness = cuckoo_search(n, dim, ReliefFList[0][1], X, y, fitness_function=relieff_eval)
CKRFFeatures = X.columns[best_nest.astype(bool)].tolist()
ReliefOpt.append([CKRFFeatures, best_CKRFfitness])


In [ ]:
print(ReliefOpt)

In [ ]:
print("Features selected by ReliefF: ", CKRFFeatures)

### Mutual Information Classifiers

#### Naive Bayes with Mutual Information

In [ ]:
NB = GaussianNB()

In [ ]:
NB.fit(X_train[CKMIFeatures], y_train)

y_pred = NB.predict(X_test[CKMIFeatures])

In [ ]:

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
add_result("Naive Bayes", "Mutual Information", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred))

### Random Forest with Mutual Information

In [ ]:
DT = RandomForestClassifier()


In [ ]:
DT.fit(X_train[CKMIFeatures], y_train)
y_pred = DT.predict(X_test[CKMIFeatures])


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
add_result("Random Forest", "Mutual Information", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred))

### Neural Network with Mutual Information

In [ ]:
History = RN.fit(X_train[CKMIFeatures], y_train, epochs=100, batch_size=32, validation_split=0.2)

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = RN.evaluate(X_test[CKMIFeatures], y_test)

print(f'Test Accuracy: {accuracy}')
# Realizar predicciones en el conjunto de prueba
y_pred_prob = RN.predict(X_test[CKMIFeatures])  # Predicciones como probabilidades
y_pred = (y_pred_prob > 0.5).astype(int)  # Convertir probabilidades a etiquetas (0 o 1)

In [ ]:
# Calcular y mostrar las métricas
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_prob))  # Usamos probabilidades para AUC
add_result("Neural Network", "Mutual Information", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred_prob))

### Chi2 Classifiers

#### Naive Bayes with chi2

In [ ]:
NB.fit(X_train[CKX2Features], y_train)
y_pred = NB.predict(X_test[CKX2Features])


In [ ]:

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
add_result("Naive Bayes", "X2", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred))

#### Random Forest with chi2

In [ ]:
DT.fit(X_train[CKX2Features], y_train)
y_pred = DT.predict(X_test[CKX2Features])

In [ ]:

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
add_result("Random Forest", "X2", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred))

#### Neural Network with chi2

In [ ]:
history = RN.fit(X_train[CKX2Features], y_train, epochs=100, batch_size=32, validation_split=0.2)

loss, accuracy = RN.evaluate(X_test[CKX2Features], y_test)

print(f'Test Accuracy: {accuracy}')
y_pred_prob = RN.predict(X_test[CKX2Features]) 
y_pred = (y_pred_prob > 0.5).astype(int)  

In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_prob)) 
add_result("Neural Network", "X2", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred_prob))

### ReliefF Classifiers

#### Naive Bayes with ReliefF

In [ ]:
NB.fit(X_train[CKRFFeatures], y_train)
y_pred = NB.predict(X_test[CKRFFeatures])


In [ ]:

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
add_result("Naive Bayes", "Relief", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred))

#### Random Forest with ReliefF


In [ ]:
DT.fit(X_train[CKRFFeatures], y_train)
y_pred = DT.predict(X_test[CKRFFeatures])


In [ ]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred))
add_result("Random Forest", "Relief", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred))

#### Neural Network with ReliefF

In [ ]:
history = RN.fit(X_train[CKRFFeatures], y_train, epochs=100, batch_size=32, validation_split=0.2)

loss, accuracy = RN.evaluate(X_test[CKRFFeatures], y_test)

print(f'Test Accuracy: {accuracy}')
y_pred_prob = RN.predict(X_test[CKRFFeatures]) 
y_pred = (y_pred_prob > 0.5).astype(int)  

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_pred_prob)) 
add_result("Neural Network", "Relief", accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred_prob))

# Display results

In [ ]:
display_tables()